In [1]:
# Import libraries
import numpy as np 
import pandas as pd 
import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests 
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: ...working... 
  - anaconda/win-64::ca-certificates-2020.1.1-0, anaconda/win-64::openssl-1.1.1d-he774522_4
  - anaconda/win-64::openssl-1.1.1d-he774522_4, defaults/win-64::ca-certificates-2020.1.1-0
  - anaconda/win-64::ca-certificates-2020.1.1-0, defaults/win-64::openssl-1.1.1d-he774522_4
  - defaults/win-64::ca-certificates-2020.1.1-0, defaults/win-64::openssl-1.1.1d-he774522_4done

# All requested packages already installed.

Solving environment: ...working... 
  - anaconda/win-64::ca-certificates-2020.1.1-0, anaconda/win-64::openssl-1.1.1d-he774522_4
  - anaconda/win-64::openssl-1.1.1d-he774522_4, defaults/win-64::ca-certificates-2020.1.1-0
  - anaconda/win-64::ca-certificates-2020.1.1-0, defaults/win-64::openssl-1.1.1d-he774522_4
  - defaults/win-64::ca-certificates-2020.1.1-0, defaults/win-64::openssl-1.1.1d-he774522_4done

# All requested packages already installed.



In [2]:
# Scraping table from Wiki
import pandas as pd
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table = pd.read_html(url)
df = table[0]
Index_Name = df[df["Borough"]=="Not assigned"].index
df.drop(Index_Name, inplace = True)
df = df.reset_index(drop = True)

# Reading lat-lonf for postal code
url1 = "https://cocl.us/Geospatial_data"
df1 = pd.read_csv(url1)
df1.head()

# Joining two dataframes using postal codes
df_merge = pd.merge(left = df, right = df1, how = 'left', left_on = 'Postal code',right_on = 'Postal Code')
df_merge.drop(axis=1, columns = 'Postal Code', inplace= True)
df_merge # Toronto data

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,43.636258,-79.498509


In [3]:
# Visualize neighborhoods

#Create Toronto map
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

#Superimposing neighborhoods on the map
for lat, lng, borough, neighborhood in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Borough'], df_merge['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
map_toronto

In [4]:
# Creating credentials for Foursquare
CLIENT_ID = 'CKZZZMHGKVSQPIUOEIMAXJS45CZEZ5XUD5DZEM0IIOEQJAJ4'
CLIENT_SECRET = 'N5WGEMVBWOM3FIWAEHV1O3LCITKSICXYFWH3XAZT5ZZHPRKI'
VERSION = '20180605'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CKZZZMHGKVSQPIUOEIMAXJS45CZEZ5XUD5DZEM0IIOEQJAJ4
CLIENT_SECRET:N5WGEMVBWOM3FIWAEHV1O3LCITKSICXYFWH3XAZT5ZZHPRKI


In [5]:
# Function that get venues for each neighborhood from Foursquare

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        try :
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except:
            print("")  
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
# Exploring venues (max = 50) in each neighborhood

trt_venues = getNearbyVenues(names = df_merge['Neighborhood'],
                                   latitudes=df_merge['Latitude'],
                                   longitudes=df_merge['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park / Harbourfront
Lawrence Manor / Lawrence Heights
Queen's Park / Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don Mills
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
East Toronto
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
Bayview Village
Do

In [7]:
trt_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [8]:
trt_venues.shape

(1696, 7)

In [9]:
# Finding number of unique categories

len(trt_venues['Venue Category'].unique())

251

In [10]:
# One hot encoding
trt_onehot = pd.get_dummies(trt_venues[['Venue Category']], prefix="", prefix_sep="")
trt_onehot['Neighborhood'] = trt_venues['Neighborhood'] 
trt_onehot.head()

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Move "Neighborhood" to the 1st column
trt_onehot.columns.get_loc('Neighborhood')

176

In [12]:
# Move "Neighborhood" to the 1st column
new_cols = list(trt_onehot.columns[176:])+list(trt_onehot.columns[:176])
trt_onehot = trt_onehot[new_cols]
trt_onehot.head()

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,...,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue
0,Parkwoods,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# Overview of stats
trt_grouped = trt_onehot.groupby('Neighborhood').mean().reset_index()
trt_grouped.head(10)

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,...,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue
0,Agincourt,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
1,Alderwood / Long Branch,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
2,Bathurst Manor / Wilson Heights / Downsview North,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
3,Bayview Village,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
4,Bedford Park / Lawrence Manor East,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
5,Berczy Park,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.020000,0.000000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0
6,Birch Cliff / Cliffside West,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
7,Brockton / Parkdale Village / Exhibition Place,0.0,0.045455,0.0,0.0,0.0,0.0,0.0,0.000000,0.045455,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
8,Business reply mail Processing CentrE,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.055556,0.000000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
9,CN Tower / King and Spadina / Railway Lands / ...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0


In [20]:
# Clustering
k = 5

trt_grouped_clustering = trt_grouped.drop("Neighborhood",1)
kmeans = KMeans(n_clusters=k, random_state=0).fit(trt_grouped_clustering)

kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 3, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 4, 1,
       1, 1, 1, 2, 1, 1, 0])

In [21]:
# Function that return most common venues/categories

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
# List top 10 most common venues/categories
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = trt_grouped['Neighborhood']

for ind in np.arange(trt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(trt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(15)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Clothing Store,Breakfast Spot,Skating Rink,Aquarium,Airport Service,Airport Terminal,American Restaurant,Antique Shop
1,Alderwood / Long Branch,Pizza Place,Skating Rink,Gym,Sandwich Place,Dance Studio,Athletics & Sports,Coffee Shop,Pub,Pharmacy,Music Venue
2,Bathurst Manor / Wilson Heights / Downsview North,Bank,Coffee Shop,Video Store,Pizza Place,Shopping Mall,Deli / Bodega,Ice Cream Shop,Sandwich Place,Restaurant,Supermarket
3,Bayview Village,Japanese Restaurant,Café,Chinese Restaurant,Bank,Art Museum,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery
4,Bedford Park / Lawrence Manor East,Sandwich Place,Italian Restaurant,Restaurant,Thai Restaurant,Coffee Shop,Butcher,Liquor Store,Spa,Cupcake Shop,Grocery Store
5,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Café,Italian Restaurant,Cheese Shop,Beer Bar,Farmers Market,Bakery
6,Birch Cliff / Cliffside West,General Entertainment,Café,College Stadium,Skating Rink,Art Gallery,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium
7,Brockton / Parkdale Village / Exhibition Place,Café,Coffee Shop,Breakfast Spot,Climbing Gym,Italian Restaurant,Intersection,Stadium,Grocery Store,Bar,Gym
8,Business reply mail Processing CentrE,Light Rail Station,Garden Center,Brewery,Fast Food Restaurant,Comic Shop,Yoga Studio,Restaurant,Pizza Place,Garden,Farmers Market
9,CN Tower / King and Spadina / Railway Lands / ...,Airport Service,Airport Terminal,Airport Lounge,Plane,Rental Car Location,Airport Gate,Boutique,Airport Food Court,Airport,Boat or Ferry


In [32]:
# Adding clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

trt_merged = df_merge

# Merging toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
trt_merged = trt_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
trt_merged.head(15)

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Food & Drink Shop,Construction & Landscaping,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Music Venue,Airport Lounge
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,French Restaurant,Intersection,Pizza Place,Portuguese Restaurant,Coffee Shop,Hockey Arena,Athletics & Sports,Asian Restaurant,Arts & Crafts Store,Art Museum
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,1.0,Coffee Shop,Bakery,Pub,Park,Theater,Café,Breakfast Spot,Restaurant,Art Gallery,Gym / Fitness Center
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,1.0,Clothing Store,Women's Store,Event Space,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Gift Shop,Miscellaneous Shop,Boutique,Coffee Shop
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,1.0,Coffee Shop,Sushi Restaurant,Diner,Discount Store,Italian Restaurant,Sandwich Place,Hobby Shop,Gym,Spa,Theater
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,3.0,Fast Food Restaurant,Music Venue,Airport Lounge,Auto Workshop,Auto Garage,Athletics & Sports,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery
7,M3B,North York,Don Mills,43.745906,-79.352188,1.0,Gym,Sporting Goods Shop,Japanese Restaurant,Restaurant,Coffee Shop,Asian Restaurant,Beer Store,Caribbean Restaurant,Bike Shop,Athletics & Sports
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937,1.0,Pizza Place,Fast Food Restaurant,Athletics & Sports,Bank,Gym / Fitness Center,Intersection,Gastropub,Pet Store,Pharmacy,Aquarium
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.0,Coffee Shop,Café,Theater,Restaurant,Ramen Restaurant,Middle Eastern Restaurant,Clothing Store,Bookstore,Tea Room,Sandwich Place


In [33]:
# Filtering only "Toronto"
trt_final= trt_merged[trt_merged['Neighborhood'].str.contains("Toronto", na=False)].reset_index()
trt_final

,index,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,35,M4J,East York,East Toronto,43.685347,-79.338106,0.0,Park,Convenience Store,Music Venue,Airport Lounge,Auto Workshop,Auto Garage,Athletics & Sports,Asian Restaurant,Arts & Crafts Store,Art Museum
1,36,M5J,Downtown Toronto,Harbourfront East / Union Station / Toronto Is...,43.640816,-79.381752,1.0,Coffee Shop,Aquarium,Hotel,Plaza,Park,Café,Brewery,IT Services,Salad Place,History Museum
2,42,M5K,Downtown Toronto,Toronto Dominion Centre / Design Exchange,43.647177,-79.381576,1.0,Café,Coffee Shop,Seafood Restaurant,Japanese Restaurant,Hotel,Restaurant,Sporting Goods Shop,General Travel,Basketball Stadium,Speakeasy
3,73,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1.0,Clothing Store,Coffee Shop,Café,Diner,Bagel Shop,Spa,Salon / Barbershop,Sporting Goods Shop,Gym / Fitness Center,Restaurant
4,80,M5S,Downtown Toronto,University of Toronto / Harbord,43.662696,-79.400049,1.0,Café,Bookstore,Bakery,Italian Restaurant,Japanese Restaurant,Restaurant,Bar,Video Game Store,Sushi Restaurant,Noodle House
5,88,M8V,Etobicoke,New Toronto / Mimico South / Humber Bay Shores,43.605647,-79.501321,1.0,Café,Pet Store,Fast Food Restaurant,American Restaurant,Gym,Seafood Restaurant,Restaurant,Liquor Store,Pizza Place,Pharmacy


In [34]:
# droping some colums

# We will find 6 neighborhoods related to "Toronto", top 10 most common venues and cluster labels of the those negihborhoods

trt_final.drop(columns = ["index","Postal code","Borough"])

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,43.685347,-79.338106,0.0,Park,Convenience Store,Music Venue,Airport Lounge,Auto Workshop,Auto Garage,Athletics & Sports,Asian Restaurant,Arts & Crafts Store,Art Museum
1,Harbourfront East / Union Station / Toronto Is...,43.640816,-79.381752,1.0,Coffee Shop,Aquarium,Hotel,Plaza,Park,Café,Brewery,IT Services,Salad Place,History Museum
2,Toronto Dominion Centre / Design Exchange,43.647177,-79.381576,1.0,Café,Coffee Shop,Seafood Restaurant,Japanese Restaurant,Hotel,Restaurant,Sporting Goods Shop,General Travel,Basketball Stadium,Speakeasy
3,North Toronto West,43.715383,-79.405678,1.0,Clothing Store,Coffee Shop,Café,Diner,Bagel Shop,Spa,Salon / Barbershop,Sporting Goods Shop,Gym / Fitness Center,Restaurant
4,University of Toronto / Harbord,43.662696,-79.400049,1.0,Café,Bookstore,Bakery,Italian Restaurant,Japanese Restaurant,Restaurant,Bar,Video Game Store,Sushi Restaurant,Noodle House
5,New Toronto / Mimico South / Humber Bay Shores,43.605647,-79.501321,1.0,Café,Pet Store,Fast Food Restaurant,American Restaurant,Gym,Seafood Restaurant,Restaurant,Liquor Store,Pizza Place,Pharmacy
